# Training Deep Neural Networks

So far we trained shallow nets, which have few hidden layers. Now, if we have to tackle more complex problems like, object detection or speech recognition. Then we have increase our layers and neurons. But training a DNN is not that easy, there are several problems that can occur:

- *Vanishing/Exploding gradients* problem: this is when gradients grow smaller and smaller, or bigger and bigger during back propagation. This will make lower layers hard to train.
- You might not have enough data, or label the data is too costly.
- Training may be extremely slow.
- Model could easily overfit the data having too many parameters.

So, let's tackle these problems. Welcome to Deep Learning!

# Vanishing/Exploding Gradients Problem

In backpropagation gradients often get smaller and smaller when getting to lower layers. Hence, updates of parameters of those layers are negligible, this is called *vanishing gradients*. On other hand sometimes gradients grow bigger and bigger and updates on parameters are too large leading algorithm to diverge. This is called *exploding gradients*, it usually occurs in RNN.

The reason of vanishing gradients are activation functions used. Like in sigmoid functions when bigger values occur it tends to 0 or 1, where the gradients is almost zero or very minimum. So backpropagation keeps diluting the gradients.

## Glorot and He Initialization

Xavier Glorot and Yoshua Bengio proposed a paper to solve this problem by pointing out that we need the signal to flow properly in both directions (forward and back propagation): means the variance of inputs and outputs of a layer should be equal same goes for the gradients.

So they propose that we should initialize weights randomly as:
$$
fan_{avg} = (fan_{in} + fan_{out}) / 2
\\
\sigma^2 = \frac1{fan_{avg}} , mean=0: for\ normal\ distribution
\\
r = \sqrt{\frac3{fan_{avg}}}, (-r, +r): for\ uniform\ distribution
$$
Here, $fan_{in}$ and $fan_{out}$ are number of inputs and neurons of a layer.

This is called *Xavier Initialization* or *Glorot Initialization*. If you replace $fan_{avg}$ to $fan_{in}$ you get *LeCun Initialization*, which was proposed back in 1990. These initialization can be used with logistic activation functions.

For ReLU (and its variance), there is a *He Initialization*. Where $\sigma^2=\frac2{fan_{in}}$.

By default, Keras uses Glorot Initialization with uniform distribution. When creating a layer you can change this to He initialization by setting `kernel_initializer='he_uniform'` or `kernel_initializer='he_normal'` like this:

```python
keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')
```

If you want to use $fan_{avg}$ in He initialization with uniform distribution, you can use `VarianceScaling` like this:

```python
he_avg_init = keras.initializers.VarianceScaling(scale=2, mode='fan_avg',
                                                distribution='uniform')
keras.layers.Dense(10, activation='relu', kernel_initializer=he_avg_init)
```

## Non-saturating Activation Functions

Earlier we see that the problems with unstable gradients are due to poor choice of activation function. So, let's change it as nowadays use mostly ReLU, it does not saturates for the positive values (and it is fast). 

Unfortunately, ReLU suffers from a problem called *dying ReLU*: during training some neurons effectively "die", meaning they stop outputting anything other than 0.

To solve this problem, there is a variant called *Leaky ReLU*. It is defined as:
$$
LeakyReLU_\alpha(z) = max(\alpha z, z)
$$
where $\alpha$ defines how much slope in z < 0, and typically set to 0.01. 

There are other variants proposed in 2015, $\alpha=0.2$ is better than 0.01, *randomized Leaky ReLU* (chooses $\alpha$ randomly), and *parametric Leaky ReLU* (trains value of $\alpha$).

A 2015 paper also proposed a new activation function called *exponential linear unit (ELU)*:
$$
ELU_\alpha(z) =
\begin{cases}
    \alpha(\exp(z)-1),& if\ z < 0\\
    z,              & if\ z \ge 0
\end{cases}
$$
It solves dead neuron problem, if $\alpha=1$, then it is smooth everywhere which helps in GD.

The main drawback is it is slower to compute, so it does reduce training time but computes slow which results similar time as using ReLU. 

In 2017 paper, they introduces a *scaled ELU*, they showed if you build a network with stack of dense layers, and if all hidden layers uses SELU, then the network will be *self-normalize*. Means output of each layer tends to preserve mean 0 and std of 1 during training. This outperforms all other activation functions in such networks.

However, there are conditions for self-normalization to happen:

- Input features must be standardized (mean 0 and std 1).
- Initialization must be LeCun normal, this means setting `kernel_initializer='lecun_normal'`.
- The network must be sequential. Not RNN and skip connections.
- All layers must be dense, but in some cases they perform better on CNN also.

> ReLU is default in most cases because it is faster to calculate than others. Which increases the latency of network and nowadays latency is important than faster training or convergence.

To use Leaky ReLU:

```python
model = keras.models.Sequential([
    [...]
    keras.layers.Dense(10),
    keras.layers.LeakyReLU(alpha=0.2),
    [...]
])
```

For SELU:

```python
layer = keras.layers.Dense(10, activation='selu', 
                          kernel_initializer='lecun_normal')
```
